In [ ]:
!pip install catboost
!pip install optuna
!pip install xgboost

In [2]:
import numpy as np
import pandas as pd
import random
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedKFold # StratifiedKFold
from catboost import  CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# Optuna
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
# Seed 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42)

In [6]:
x_train = pd.read_csv('/content/drive/MyDrive/uou-g03784-02-spring-2023-term-project/x_train.csv')
y_train = pd.read_csv('/content/drive/MyDrive/uou-g03784-02-spring-2023-term-project/y_train.csv')
test = pd.read_csv('/content/drive/MyDrive/uou-g03784-02-spring-2023-term-project/x_test.csv')
submission = pd.read_csv('/content/drive/MyDrive/uou-g03784-02-spring-2023-term-project/sample_submission.csv')

In [7]:
y_train = y_train['Rented Bike Count']

In [8]:
drop_features = ['ID']
x_train = x_train.drop(drop_features, axis=1)
test = test.drop(drop_features, axis=1)
x_train.head(3)

,Hour,Temperature,Humidity,Wind speed,Visibility,Dew point temperature,Solar Radiation,Rainfall,Snowfall,Seasons_Autumn,Seasons_Spring,Seasons_Summer,Seasons_Winter,Holiday,Functional Hours
0,6,-15.7,29,2.5,2000,-29.7,0.00,0.0,0.9,0.0,0.0,0.0,1.0,0.0,1.0
1,15,6.8,50,4.1,660,-2.9,0.56,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,6,1.1,66,0.9,1550,-4.5,0.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [9]:
lgbm_reg = LGBMRegressor(n_estimators=260,learning_rate = 0.05, num_leaves=50, random_state = 0)
lgbm_reg.fit(x_train, y_train)
prediction = lgbm_reg.predict(x_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001502 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1257
[LightGBM] [Info] Number of data points in the train set: 7008, number of used features: 15
[LightGBM] [Info] Start training from score 699.971747


In [10]:
# 절댓값 잔차를 계산합니다.
residuals = np.abs(prediction - y_train)

In [11]:
# 잔차가 500보다 큰 데이터의 인덱스를 찾습니다.
outliers = np.where(residuals > 490)[0]

In [12]:
len(outliers)

64

In [13]:
# 이상치를 제거한 새로운 x_train과 y_train을 만듭니다.
x_train_cleaned = x_train.drop(outliers, axis=0).reset_index(drop=True)
y_train_cleaned = y_train.drop(outliers, axis=0).reset_index(drop=True)

In [14]:
X_train, X_val, Y_train, Y_val = train_test_split(x_train_cleaned, y_train_cleaned, test_size=0.2, shuffle = True, random_state = 42)

In [15]:
# random sampler
sampler = TPESampler(seed=42)

# define function
def objective(trial):

    cbrm_param = {
        'objective' :  'RMSE',
        'n_estimators':trial.suggest_int("n_estimators", 100, 500),
        # 'iterations':trial.suggest_int("iterations", 30, 400),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'subsample': trial.suggest_uniform('subsample',0.1,1),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
    }
    # Generate model
    model_cbrm = CatBoostRegressor(**cbrm_param)
    model_cbrm = model_cbrm.fit(X_train, Y_train, eval_set=[(X_val, Y_val)],
                           verbose=0, early_stopping_rounds=25)

    # 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(Y_val, model_cbrm.predict(X_val)))
    return RMSE

In [16]:
optuna_cbrm = optuna.create_study(direction='minimize', sampler=sampler)
optuna_cbrm.optimize(objective, n_trials = 100)

[I 2024-05-28 14:34:06,324] A new study created in memory with name: no-name-ce5e085e-df65-453a-a108-67a7c34fa3ad
[I 2024-05-28 14:34:06,568] Trial 0 finished with value: 307.8701453987418 and parameters: {'n_estimators': 250, 'od_wait': 2212, 'learning_rate': 0.7346740023932911, 'reg_lambda': 59.86585243311882, 'random_strength': 16.24074561769746, 'subsample': 0.2403950683025824, 'depth': 1, 'min_data_in_leaf': 26}. Best is trial 0 with value: 307.8701453987418.
[I 2024-05-28 14:34:06,925] Trial 1 finished with value: 371.164676410536 and parameters: {'n_estimators': 341, 'od_wait': 1775, 'learning_rate': 0.03037864935284442, 'reg_lambda': 96.99098551710091, 'random_strength': 43.29770563201687, 'subsample': 0.29110519961044856, 'depth': 3, 'min_data_in_leaf': 6}. Best is trial 0 with value: 307.8701453987418.
[I 2024-05-28 14:34:07,236] Trial 2 finished with value: 213.33684532959012 and parameters: {'n_estimators': 222, 'od_wait': 1445, 'learning_rate': 0.4376255684556946, 'reg_lam

In [17]:
cbrm_trial = optuna_cbrm.best_trial
cbrm_trial_params = cbrm_trial.params
print('Best Trial: score {},\nparams {}'.format(cbrm_trial.value, cbrm_trial_params))

Best Trial: score 195.24590464824124,
params {'n_estimators': 457, 'od_wait': 2061, 'learning_rate': 0.22490839688594771, 'reg_lambda': 15.411129645304804, 'random_strength': 41.03095376926677, 'subsample': 0.933755587364059, 'depth': 11, 'min_data_in_leaf': 20}


In [18]:
cbr = CatBoostRegressor(**cbrm_trial_params, verbose=100)

#최적 하이퍼파라미터로 학습
cbr.fit(x_train_cleaned, y_train_cleaned, eval_set=[(X_val, Y_val)], verbose=100, early_stopping_rounds=100)

0:	learn: 598.9898267	test: 608.9058382	best: 608.9058382 (0)	total: 16.5ms	remaining: 7.54s
100:	learn: 165.0365007	test: 168.1359937	best: 168.1359937 (100)	total: 1.39s	remaining: 4.88s
200:	learn: 122.9244403	test: 126.3292763	best: 126.3292763 (200)	total: 2.87s	remaining: 3.65s
300:	learn: 98.9530471	test: 102.4042391	best: 102.4042391 (300)	total: 4.38s	remaining: 2.27s
400:	learn: 83.1250870	test: 86.5540031	best: 86.5540031 (400)	total: 5.95s	remaining: 830ms
456:	learn: 75.6884856	test: 78.9928747	best: 78.9928747 (456)	total: 6.79s	remaining: 0us

bestTest = 78.99287466
bestIteration = 456



In [19]:
# define function
def objective(trial):

    xgbr_param = {
        'metric': 'rmse',
        'lambda': trial.suggest_float('lambda', 1e-3, 0.1),
        'alpha': trial.suggest_float('alpha', 1e-3, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1),
        'subsample': trial.suggest_float('subsample', 0.4, 1),
        'learning_rate': trial.suggest_float('learning_rate',0.0001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 4,10),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 50),
    }
    # Generate model
    model_xgbr = XGBRegressor(**xgbr_param)
    model_xgbr = model_xgbr.fit(X_train, Y_train, eval_set=[(X_val, Y_val)],
                           verbose=0, early_stopping_rounds=25)

    # 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(Y_val, model_xgbr.predict(X_val)))
    return RMSE

In [20]:
optuna_xgbr = optuna.create_study(direction='minimize', sampler=sampler)
optuna_xgbr.optimize(objective, n_trials = 100)

[I 2024-05-28 14:56:22,130] A new study created in memory with name: no-name-ca44f059-3242-46c5-b1d0-4056150487af
[I 2024-05-28 14:56:22,680] Trial 0 finished with value: 216.36095133846715 and parameters: {'lambda': 0.08644723916168376, 'alpha': 0.6236748287007303, 'colsample_bytree': 0.5985388149115896, 'subsample': 0.4381350101716142, 'learning_rate': 0.031167133939394655, 'n_estimators': 230, 'max_depth': 9, 'min_child_weight': 33}. Best is trial 0 with value: 216.36095133846715.
[I 2024-05-28 14:56:23,309] Trial 1 finished with value: 208.36484166180776 and parameters: {'lambda': 0.08883406151505634, 'alpha': 0.4727427102367873, 'colsample_bytree': 0.471756547562981, 'subsample': 0.827946872333797, 'learning_rate': 0.07610242635682805, 'n_estimators': 325, 'max_depth': 9, 'min_child_weight': 26}. Best is trial 1 with value: 208.36484166180776.
[I 2024-05-28 14:56:23,898] Trial 2 finished with value: 407.55305736011695 and parameters: {'lambda': 0.052750550108817414, 'alpha': 0.428

In [21]:
xgbr_trial = optuna_xgbr.best_trial
xgbr_trial_params = xgbr_trial.params
print('Best Trial: score {},\nparams {}'.format(xgbr_trial.value, xgbr_trial_params))

Best Trial: score 196.2864113520533,
params {'lambda': 0.07820240307153835, 'alpha': 0.48284493858741406, 'colsample_bytree': 0.854713803372864, 'subsample': 0.9451353161994753, 'learning_rate': 0.06077000526002691, 'n_estimators': 316, 'max_depth': 9, 'min_child_weight': 6}


In [22]:
xgbr = XGBRegressor(**xgbr_trial_params, verbose=100)

#최적 하이퍼파라미터로 학습
xgbr.fit(x_train_cleaned, y_train_cleaned, eval_set=[(X_val, Y_val)], verbose=100, early_stopping_rounds=100)

[0]	validation_0-rmse:617.42367
[100]	validation_0-rmse:105.51118
[200]	validation_0-rmse:75.94794
[300]	validation_0-rmse:59.98243
[315]	validation_0-rmse:57.78600


XGBRegressor(alpha=0.48284493858741406, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.854713803372864, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             lambda=0.07820240307153835, learning_rate=0.06077000526002691,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=6, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=316, n_jobs=None, ...)

In [29]:
# define function
def objective(trial):

    lgbr_param = {
        'objective': 'regression', # 회귀
        'metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate",0.0001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }
    # Generate model
    model_lgbr = LGBMRegressor(**lgbr_param)
    model_lgbr = model_lgbr.fit(X_train, Y_train, eval_set=[(X_val, Y_val)])

    # 평가지표 원하는 평가 지표가 있을 시 바꾸어 준다.
    RMSE = np.sqrt(mean_squared_error(Y_val, model_lgbr.predict(X_val)))
    return RMSE

In [30]:
optuna_lgbr = optuna.create_study(direction='minimize', sampler=sampler)
optuna_lgbr.optimize(objective, n_trials = 100)

[I 2024-05-28 15:00:44,017] A new study created in memory with name: no-name-689107bb-d66f-47a2-9eb7-9763f0e5e690


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000336 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1248
[LightGBM] [Info] Number of data points in the train set: 5555, number of used features: 15
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 692.354815
[LightGBM] [Warning] No further splits with

[I 2024-05-28 15:00:44,433] Trial 0 finished with value: 263.36329962811976 and parameters: {'max_depth': 5, 'learning_rate': 0.011825328508422656, 'n_estimators': 255, 'min_child_samples': 94, 'subsample': 0.45371702631829186}. Best is trial 0 with value: 263.36329962811976.


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000665 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1248
[LightGBM] [Info] Number of data points in the train set: 5555, number of used features: 15
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 692.354815


[I 2024-05-28 15:00:45,043] Trial 1 finished with value: 608.3927437748263 and parameters: {'max_depth': 7, 'learning_rate': 0.00021898812429056952, 'n_estimators': 470, 'min_child_samples': 89, 'subsample': 0.5066467777433962}. Best is trial 0 with value: 263.36329962811976.


...

[I 2024-05-28 15:01:24,703] Trial 97 finished with value: 200.2706505338769 and parameters: {'max_depth': 12, 'learning_rate': 0.05676182485779687, 'n_estimators': 373, 'min_child_samples': 17, 'subsample': 0.5156133277312547}. Best is trial 47 with value: 195.50136866698023.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001029 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1248
[LightGBM] [Info] Number of data points in the train set: 5555, number of used features: 15
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max

[I 2024-05-28 15:01:25,183] Trial 98 finished with value: 199.528174600211 and parameters: {'max_depth': 13, 'learning_rate': 0.03728665379215056, 'n_estimators': 407, 'min_child_samples': 7, 'subsample': 0.44273199013919323}. Best is trial 47 with value: 195.50136866698023.


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000606 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1248
[LightGBM] [Info] Number of data points in the train set: 5555, number of used features: 15
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training

[I 2024-05-28 15:01:25,784] Trial 99 finished with value: 205.65067867561248 and parameters: {'max_depth': 6, 'learning_rate': 0.0738138000186656, 'n_estimators': 426, 'min_child_samples': 46, 'subsample': 0.4955028541209918}. Best is trial 47 with value: 195.50136866698023.


In [31]:
lgbr_trial = optuna_lgbr.best_trial
lgbr_trial_params = lgbr_trial.params
print('Best Trial: score {},\nparams {}'.format(lgbr_trial.value, lgbr_trial_params))

Best Trial: score 195.50136866698023,
params {'max_depth': 14, 'learning_rate': 0.09947633780403212, 'n_estimators': 306, 'min_child_samples': 13, 'subsample': 0.4783248624332811}


In [33]:
lgbr = LGBMRegressor(**lgbr_trial_params, verbose=100)

#최적 하이퍼파라미터로 학습
lgbr.fit(x_train_cleaned, y_train_cleaned, eval_set=[(X_val, Y_val)])

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.951649
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.363527
[LightGBM] [Debug] init for col-wise cost 0.000572 seconds, init for row-wise cost 0.000989 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001130 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1258
[LightGBM] [Info] Number of data points in the train set: 6944, number of used features: 15
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (n

LGBMRegressor(learning_rate=0.09947633780403212, max_depth=14,
              min_child_samples=13, n_estimators=306,
              subsample=0.4783248624332811, verbose=100)

In [34]:
cbr_prediction = cbr.predict(test)

In [35]:
xgbr_prediction = xgbr.predict(test)

In [36]:
lgbr_prediction = lgbr.predict(test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [41]:
prediction = (cbr_prediction * 0.5 + xgbr_prediction * 0.2 + lgbr_prediction * 0.3)

In [42]:
submission['Rented Bike Count'] = prediction

In [43]:
submission.to_csv('submission.csv', index=False)

In [44]:
from google.colab import files

# submission.csv 파일 다운로드
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>